# Data Science Take Home Test

## Initialization

Import libraries

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time
start_time = time.time()

Load data

In [89]:
df_mes = pd.read_csv('I:\Javier Resano\Curriculum\Caso Carto\yellow_tripdata_2017-03.csv', sep=',')

## Data Exploration and Cleaning

In [90]:
df_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10294628 entries, 0 to 10294627
Data columns (total 17 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
RatecodeID               int64
store_and_fwd_flag       object
PULocationID             int64
DOLocationID             int64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 1.3+ GB


**To note:**
-  "tpep_pickup_datetime" and "tpep_dropoff_datetime" should be datetime
- PULocationID, DOLocationID, payment_type, RateCodeID are really numeric codes. I'll transform them to strings

In [91]:
df_mes['tpep_pickup_datetime'] = pd.to_datetime(df_mes['tpep_pickup_datetime'])
df_mes['tpep_dropoff_datetime'] = pd.to_datetime(df_mes['tpep_dropoff_datetime'])
df_mes['PULocationID'] = df_mes['PULocationID'].astype(str)
df_mes['DOLocationID'] = df_mes['DOLocationID'].astype(str)
df_mes['payment_type'] = df_mes['payment_type'].astype(str)
df_mes['RatecodeID'] = df_mes['RatecodeID'].astype(str)
df_mes['VendorID'] = df_mes['VendorID'].astype(str)

In [92]:
with pd.option_context('float_format', '{:.2f}'.format): print(df_mes.describe()) #10.294.628 cases

       passenger_count  trip_distance  fare_amount       extra     mta_tax  \
count      10294628.00    10294628.00  10294628.00 10294628.00 10294628.00   
mean              1.62           2.88        12.91        0.34        0.50   
std               1.26           3.72        54.99        0.47        0.07   
min               0.00           0.00      -308.00      -53.71       -0.50   
25%               1.00           0.94         6.50        0.00        0.50   
50%               1.00           1.60         9.50        0.00        0.50   
75%               2.00           3.00        14.50        0.50        0.50   
max               9.00         598.50    171861.78       69.80      103.30   

       tip_amount  tolls_amount  improvement_surcharge  total_amount  
count 10294628.00   10294628.00            10294628.00   10294628.00  
mean         1.86          0.31                   0.30         16.22  
std          2.62          1.90                   0.01         55.62  
min        -9

** Findings. Data issues: **
- trip_distance sometimes is 0 (no travel?)
- fare_amount sometimes contains negative numbers
- extra sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.5 or 1 charges)
- mta_tax sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.5 charges)
- total_amount sometimes contains negative numbers
- tip_amount sometimes contains negative numbers
- tolls_amount sometimes contains negative numbers
- improvement_surcharge sometimes contains negative numbers. Max value is strange too. Does not seem to match Data Dictionary explanation (only 0.3 charges)
   

Since we have a lot of data, I will remove wrong data. <br>
Initial number of rows:

In [93]:
df_mes.shape[0]

10294628

In [94]:
df_mes = df_mes[df_mes['trip_distance']>0]
df_mes.shape[0]

10228211

In [95]:
df_mes = df_mes[df_mes['fare_amount']>0]
df_mes.shape[0]

10222592

In [96]:
df_mes = df_mes[df_mes['extra']>=0]
df_mes.shape[0]

10222581

In [97]:
df_mes = df_mes[df_mes['mta_tax']>=0]
df_mes.shape[0]

10222581

In [98]:
df_mes = df_mes[df_mes['total_amount']>=0]
df_mes.shape[0]

10222581

In [99]:
df_mes = df_mes[df_mes['tip_amount']>=0]
df_mes.shape[0]

10222580

In [100]:
df_mes = df_mes[df_mes['tolls_amount']>=0]
df_mes.shape[0]

10222579

In [101]:
df_mes = df_mes[df_mes['improvement_surcharge']>=0]
df_mes.shape[0]

10222579

Now, I'll check and delete strange cases:

In [102]:
df_mes[(df_mes['improvement_surcharge']!=0.3)].shape[0]

96

In [103]:
df_mes = df_mes[df_mes['improvement_surcharge']==0.3]
df_mes.shape[0]

10222483

In [104]:
df_mes[(df_mes['mta_tax']!=0.5)&(df_mes['mta_tax']!=0)].shape[0]

0

In [105]:
df_mes[(df_mes['extra']!=0)&(df_mes['extra']!=0.5)&(df_mes['extra']!=1)&(df_mes['extra']!=1.5)].shape[0]

41901

In [106]:
df_mes[(df_mes['extra']!=0)&(df_mes['extra']!=0.5)&(df_mes['extra']!=1)&(df_mes['extra']!=1.5)].extra.unique()

array([3.000e-01, 8.000e-01, 4.500e+00, 1.800e+00, 7.000e+00, 3.500e+00,
       1.300e+00, 2.000e-02, 2.500e+00, 7.400e-01, 6.530e+01, 4.540e+00,
       3.000e+00, 1.400e+00, 5.554e+01, 5.000e+00, 6.000e+00, 6.000e-02,
       2.304e+01, 6.980e+01, 9.000e-01, 7.000e-01, 1.000e+01])

In [107]:
df_mes = df_mes[(df_mes['extra']==0)|(df_mes['extra']==0.5)|(df_mes['extra']==1)|(df_mes['extra']==1.5)]
df_mes.shape[0]

10180582

All in all we have gone from 10294628 to 10180582 rows. We are keeping 98.9% of the initial dataset

### Feature Engineering
In order to improve dataset quality, I am going to create some new features based on the existing ones

1. Travel time

In [108]:
df_mes['travel_time'] = df_mes['tpep_dropoff_datetime'] - df_mes['tpep_pickup_datetime']
df_mes['travel_time'] = df_mes['travel_time'].dt.total_seconds()

2. Average speed

In [109]:
df_mes['average_speed'] = df_mes['trip_distance'] / df_mes['travel_time'] *3600

3. Hour of the day

In [110]:
df_mes['tpep_pickup_hour'] = pd.DatetimeIndex(df_mes['tpep_pickup_datetime']).hour
df_mes['tpep_dropoff_hour'] = pd.DatetimeIndex(df_mes['tpep_dropoff_datetime']).hour

4. Day of the week

In [111]:
df_mes['tpep_pickup_weekday'] = pd.DatetimeIndex(df_mes['tpep_pickup_datetime']).weekday
df_mes['tpep_dropoff_weekday'] = pd.DatetimeIndex(df_mes['tpep_dropoff_datetime']).weekday

And we check their values:

In [112]:
with pd.option_context('float_format', '{:.2f}'.format): print(df_mes.loc[:,['travel_time','average_speed']].describe()) #10.294.628 cases

       travel_time  average_speed
count  10180582.00    10180582.00
mean        971.86            inf
std        3143.45            nan
min     -256817.00          -8.30
25%         399.00           7.20
50%         669.00           9.76
75%        1099.00          13.29
max       97887.00            inf


Some new issues with data arise:
- There are some negative travel_time values
- "average_speed == inf" happens when travel_time is 0. Which mean there was no trip
- Very high average speeds (> 100 mph) also are a mark of wrong data

In [113]:
df_mes[(df_mes['average_speed'] == np.inf) | (df_mes['average_speed'] > 100)]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
14,2,2017-03-01 00:00:00,2017-03-01 00:00:00,5,1.86,1,N,132,50,1,...,2.97,0.00,0.3,22.77,0.0,inf,0,0,2,2
15,2,2017-03-01 00:00:00,2017-03-01 00:00:00,5,9.88,1,N,138,25,1,...,7.66,0.00,0.3,45.96,0.0,inf,0,0,2,2
150,1,2017-03-01 00:00:49,2017-03-01 00:01:48,1,14.80,5,N,76,76,1,...,0.03,0.00,0.3,30.33,59.0,9.030508e+02,0,0,2,2
1402,1,2017-03-10 14:13:28,2017-03-10 14:13:30,1,0.70,1,N,162,162,3,...,0.00,0.00,0.3,3.30,2.0,1.260000e+03,14,14,4,4
1867,1,2017-03-10 14:15:05,2017-03-10 14:15:18,1,2.90,1,N,236,236,2,...,0.00,0.00,0.3,3.30,13.0,8.030769e+02,14,14,4,4
2621,1,2017-03-10 14:17:47,2017-03-10 14:17:51,1,14.80,1,N,13,13,4,...,0.00,0.00,0.3,3.30,4.0,1.332000e+04,14,14,4,4
3206,1,2017-03-10 14:19:52,2017-03-10 14:20:05,1,4.60,1,N,237,237,4,...,0.00,0.00,0.3,3.30,13.0,1.273846e+03,14,14,4,4
3207,1,2017-03-10 14:19:52,2017-03-10 14:19:56,1,14.80,1,N,13,13,4,...,0.00,0.00,0.3,3.30,4.0,1.332000e+04,14,14,4,4
3743,1,2017-03-10 14:21:39,2017-03-10 14:21:41,1,4.60,1,N,237,237,1,...,15.00,0.00,0.3,18.30,2.0,8.280000e+03,14,14,4,4
7781,1,2017-03-10 14:35:35,2017-03-10 14:35:38,1,16.50,1,N,261,261,3,...,0.00,0.00,0.3,3.30,3.0,1.980000e+04,14,14,4,4


In [114]:
df_mes = df_mes[(df_mes['average_speed'] != np.inf) & (df_mes['average_speed'] < 100)]
df_mes.shape[0]

10171489

In [115]:
#I convert Timedelta value to float (number of seconds of the trip)
df_mes['travel_time'] = df_mes['travel_time'].astype('timedelta64[s]').dt.total_seconds()

In [116]:
df_mes[df_mes['travel_time'] < 0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
4890789,1,2017-03-14 15:15:22,2017-03-11 15:55:05,1,11.7,1,Y,164,26,1,...,3.00,5.54,0.3,52.34,-256817.0,-0.164008,15,15,1,5
7012746,1,2017-03-25 20:21:21,2017-03-25 20:05:27,1,2.2,1,N,229,50,1,...,3.25,0.00,0.3,19.55,-954.0,-8.301887,20,20,5,5


In [117]:
df_mes = df_mes[df_mes['travel_time'] > 0]
df_mes.shape[0]

10171487

All in all we have gone from 10294628 to 10171487 rows. We are still keeping **98.8% of the initial dataset**

### Filling out code variables
In order to encode all code variables (i.e. those that take string values, not numeric), and not miss any column if in the dataset a variable doesn't have a certain code, I am adding at the end of the dataframe new rows with all the vaules for these code varibles. I will remove these rows from the dataframe once they are encoded.

In [118]:
print(sorted(df_mes['payment_type'].unique()))

['1', '2', '3', '4']


In [119]:
print(sorted(df_mes['RatecodeID'].unique()))

['1', '2', '3', '4', '5', '6', '99']


In [120]:
df_mes['PULocationID'].unique().shape

(260,)

In [121]:
df_mes[df_mes['PULocationID'].astype('int64') >= 266]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday


In [122]:
df_mes['DOLocationID'].unique().shape

(262,)

In [123]:
df_mes[df_mes['DOLocationID'].astype('int64') >= 266]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday


All in all, we see that "payment_type" is mising 2 values (5 & 6), the locations also miss some out of the 265 listed in the document, and "RatecodeID" has an extra one: 99. As it is unknown for us, we wil get rid of the values (we can see there are only 3 of them)

In [124]:
df_mes[df_mes['RatecodeID'] == '99']

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,travel_time,average_speed,tpep_pickup_hour,tpep_dropoff_hour,tpep_pickup_weekday,tpep_dropoff_weekday
3308765,1,2017-03-08 19:44:56,2017-03-08 19:51:49,1,0.8,99,N,143,142,2,...,0.0,0.0,0.3,7.8,413.0,6.973366,19,19,2,2
6694366,1,2017-03-25 18:29:15,2017-03-25 18:35:35,2,1.6,99,N,145,112,1,...,1.0,0.0,0.3,9.3,380.0,15.157895,18,18,5,5
6862863,1,2017-03-20 14:00:33,2017-03-20 14:14:18,1,1.7,99,N,161,263,2,...,0.0,0.0,0.3,10.8,825.0,7.418182,14,14,0,0


In [125]:
df_mes = df_mes[df_mes['RatecodeID'] != '99']

## Data Summary
It is elaborated in a sepparate document as requested

In [126]:
#df_mes.to_csv(r'I:\Javier Resano\Curriculum\Caso Carto\March_data.txt', encoding='utf-8', index=False, sep='\t')

## Model Building

We aim to estimate the tip amount for each trip. Therefore we are facing a regression problem (rather than a classification problem, which would be if, for example we were trying to guess whether client would leave a tip or not).

In this case, we have a lot of data to train our algorithm, which we will try to put to a good use.

First thing will be to decide which model to use. We will do a quick comparison of models to wee which one performs best:

In [127]:
# Due to memory issues, I'll work with a smaller dataset. I'll use the rest to check the model.
df_mes2 = df_mes.sample(n=1000000, random_state=0) #.iloc[0:1000000,:]

In [128]:
shape1 = df_mes2.shape[0]
df2 = pd.DataFrame([['1'], ['2']], columns=(['VendorID']))
df3 = [str(x) for x in range(1,266)]
df3 = pd.DataFrame({'PULocationID':df3, 'DOLocationID':df3})
df4 = pd.DataFrame([['Y'], ['N']], columns=(['store_and_fwd_flag']))
df5 = [str(x) for x in range(1,7)]
df5 = pd.DataFrame({'payment_type':df5, 'RatecodeID':df5})

df_mes2 = df_mes2.append(df2, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df3, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df4, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df5, sort=False, ignore_index=True)

df_mes2.fillna(1, inplace=True) #1 is a value every string column has, so I am not adding new values here

In [129]:
df_mes2 = df_mes2.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
df_mes2 = pd.get_dummies(df_mes2, drop_first=True)

In [130]:
df_mes2 = df_mes2.head(shape1)

In [131]:
cols = list(df_mes2.columns)
cols.remove('tip_amount')

In [132]:
#Divide the dataset into input variables and output 
# (I get a memory error if I work with the whole set, so I'm reducing it to 1.000.000 rows)
X = df_mes2.loc[0:1000000,cols]
Y = df_mes2.loc[0:1000000,['tip_amount']]

In [133]:
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

### Model: Linear Regression

In [134]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, Y_train)

Y_lin_reg = lin_reg.predict(X_test)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, Y_lin_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y_test, Y_lin_reg))

RMSE: 0.097313694025138
Variance explained:  0.9984063242979634


This model explains 99.8% of the variance, and predicts the tip with a precission of around 9.7 cents (RMSE)

### Model: SVR
Even with the 1000000 rows reduced dataset, training this model caused memory issues.

### Model: Random Forest

In [135]:
from sklearn.ensemble import RandomForestRegressor
RF_reg = RandomForestRegressor(n_estimators=10, random_state=0)
RF_reg.fit(X_train, Y_train.values.ravel())

Y_RF_reg = RF_reg.predict(X_test)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, Y_RF_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y_test, Y_RF_reg))

RMSE: 0.3255731513476452
Variance explained:  0.9821618059346461


### Model: XGBoost

In [136]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor()
xg_reg.fit(X_train, Y_train)

Y_xg_reg = xg_reg.predict(X_test)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, Y_xg_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y_test, Y_xg_reg))


RMSE: 0.8560065682394397
Variance explained:  0.8766871397643773


### Choice:
Given the results above, the best model for the task at hand is clearly the Linear Regression model. Its virtues are:
- It provides a great accuracy, much better than the Random forest model or the XGBoost one
- It is fast to train, much more than SVR (which, with the resources at my disposal, didn't finish training), and still a bit faster than Random Forest or XGBoost
- Provides easy interpretability of the results.

### Metrics
The metrics used for this task are two:
- RMSE: Measures the **average money our prediction deviates from the real tip**. The closer to 0, the better, as this would mean that we are able to guess exactly thetip_amount
- Variance explained: How much of the variance current model explains. The closer to 1, the better, as it would mean that the model is able to explain all variance.

### Limitations and caveats
The model defined with this document uses all data provided by the taxi dataset. But the whole data is not always available. Most clearly, data like "tpep_dropoff_datetime", used to calculate "average_speed" and "travel_time" is not going to be known until the end of the trip, which limits the model usefulness.

Also, current hardware has limited the training of the model to only 1000000 rows of one month. Increasing this number meant memory issues arised and computation stopped.

### Future improvements
Another similar study could be carried out to forecast "tip_amount" based on data known at the moment of the start of the trip. Such a model, if accurate, could be used to take decisions on which passenger to choose if several are available for the car.

Feeding more data into the training of current model can also lead to better model, which reduces even further the prediction error, although at the moment it seems quite low and not much improvement should be expected.

Also, month of the year could be a parameter that affects the tip_amount. The study could be expanded to include data from all months, and using the month of the year as a parameter of the model.

### Turn into an API
Turning this model into an API shouldn't be too complicated. The tasks to would be:
- Extracting the coefficients from the model (see the last two lines in this script to have a look at the coefficients for each variable, and the independent coefficient)
- Assuming the input format for the API is the same as a row of taxi dataset, we would need to apply the necessary conversions to these data (convert data to the right formats, calculate the new variables from input data, and apply encodings to the variables)
- Then, it is just a matter of multiplying both vectors to get the predicted tip_amount as the output

In [137]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1562.70978474617 seconds ---


In [138]:
start_time = time.time()

In [139]:
df_mes2 = df_mes.iloc[5000000:6000000,:]

shape1 = df_mes2.shape[0]
df2 = pd.DataFrame([['1'], ['2']], columns=(['VendorID']))
df3 = [str(x) for x in range(1,266)]
df3 = pd.DataFrame({'PULocationID':df3, 'DOLocationID':df3})
df4 = pd.DataFrame([['Y'], ['N']], columns=(['store_and_fwd_flag']))
df5 = [str(x) for x in range(1,7)]
df5 = pd.DataFrame({'payment_type':df5, 'RatecodeID':df5})

df_mes2 = df_mes2.append(df2, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df3, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df4, sort=False, ignore_index=True)
df_mes2 = df_mes2.append(df5, sort=False, ignore_index=True)

df_mes2.fillna(1, inplace=True) #1 is a value every string column has, so I am not adding new values here

df_mes2 = df_mes2.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)
df_mes2 = pd.get_dummies(df_mes2, drop_first=True)

df_mes2 = df_mes2.head(shape1)

cols = list(df_mes2.columns)
cols.remove('tip_amount')

#Divide the dataset into input variables and output 
# (I get a memory error if I work with the whole set, so I'm reducing it to 1.000.000 rows)
X2 = df_mes2.loc[:,cols]
Y2 = df_mes2.loc[:,['tip_amount']]



In [140]:
Y_lin_reg = lin_reg.predict(X2)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y2, Y_lin_reg)))
print('Variance explained: ', metrics.explained_variance_score(Y2, Y_lin_reg))

RMSE: 0.12610211796460177
Variance explained:  0.9978274379630873


In [141]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 256.8916094303131 seconds ---


In [142]:
%store lin_reg

Stored 'lin_reg' (LinearRegression)


In [143]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(lin_reg.coef_))], axis = 1)

In [144]:
with pd.option_context('float_format', '{:.6f}'.format): print(coefficients)

                         0            0
0          passenger_count    -0.000056
1            trip_distance    -0.000557
2              fare_amount    -0.995332
3                    extra    -0.994108
4                  mta_tax    -0.991130
5             tolls_amount    -0.995578
6    improvement_surcharge 71356.532994
7             total_amount     0.995687
8              travel_time     0.000000
9            average_speed     0.000150
10        tpep_pickup_hour     0.000130
11       tpep_dropoff_hour    -0.000051
12     tpep_pickup_weekday     0.000529
13    tpep_dropoff_weekday    -0.000300
14              VendorID_1  -399.314084
15              VendorID_2  -399.322046
16            RatecodeID_1   -27.577704
17            RatecodeID_2   -27.562283
18            RatecodeID_3   -27.559757
19            RatecodeID_4   -27.571375
20            RatecodeID_5   -27.558515
21            RatecodeID_6   -27.584066
22    store_and_fwd_flag_N   -10.450072
23    store_and_fwd_flag_Y   -10.448734


In [145]:
Indep_coef = lin_reg.intercept_

In [146]:
Indep_coef

array([-20770.6391838])